# 🚀 PixelOS Build for Xaga (POCO X4 GT / Redmi K50i / Redmi Note 11T Pro+)

This notebook will guide you through building **PixelOS (sixteen-qpr1)** for the Xaga device.

## ⚠️ Prerequisites
- Google Colab Pro/Pro+ recommended (for RAM and runtime)
- Connect Google Drive for storage (output will be stored there)
- Ensure you have a stable internet connection

## 📋 Build Steps Overview
1. Setup environment & install dependencies
2. Initialize repo and sync PixelOS source
3. Clone device trees, vendor trees, kernel, and hardware repos
4. Apply required patches
5. Build the ROM

---
## 📁 Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

---
## 🔧 Step 2: Install Required Dependencies

In [ ]:
%%bash
# Update and install required packages
sudo apt-get update -y
sudo apt-get install -y \
    bc bison build-essential ccache curl flex g++-multilib gcc-multilib git \
    git-lfs gnupg gperf imagemagick lib32ncurses5-dev lib32readline-dev \
    lib32z1-dev libelf-dev liblz4-tool libncurses5 libncurses5-dev \
    libsdl1.2-dev libssl-dev libxml2 libxml2-utils lzop pngcrush rsync \
    schedtool squashfs-tools xsltproc zip zlib1g-dev python3 python-is-python3 \
    openjdk-17-jdk repo

# Initialize git-lfs
git lfs install

echo "✅ Dependencies installed successfully!"

---
## 👤 Step 3: Configure Git Identity

In [ ]:
%%bash
# Configure git identity (required for repo sync)
git config --global user.email "builder@colab.local"
git config --global user.name "Colab Builder"
git config --global color.ui false

echo "✅ Git configured!"

---
## 📂 Step 4: Create Build Directory & Initialize Repo

In [ ]:
%%bash
# Create and navigate to build directory
mkdir -p /content/pos
cd /content/pos

# Initialize PixelOS repo
repo init -u https://github.com/PixelOS-AOSP/android_manifest.git -b sixteen-qpr1 --git-lfs --depth=1

echo "✅ Repo initialized!"

---
## 🔄 Step 5: Sync PixelOS Source (This takes a LONG time ~2-4 hours)

In [ ]:
%%bash
cd /content/pos

# Sync the repo with optimizations
repo sync -c --no-tags --no-clone-bundle --optimized-fetch --prune --force-sync -j$(nproc --all)

echo "✅ PixelOS source synced!"

---
## 📱 Step 6: Clone Device & Vendor Trees

In [ ]:
%%bash
cd /content/pos

echo "📱 Cloning Device Trees..."

# Device Tree (xaga)
git clone https://github.com/XagaForge/android_device_xiaomi_xaga.git device/xiaomi/xaga --depth=1

# Device Tree (mt6895-common)
git clone https://github.com/XagaForge/android_device_xiaomi_mt6895-common.git device/xiaomi/mt6895-common --depth=1

echo "✅ Device trees cloned!"

echo "📦 Cloning Vendor Trees..."

# Vendor Tree (xaga)
git clone https://gitlab.com/priiii08918/android_vendor_xiaomi_xaga.git vendor/xiaomi/xaga --depth=1

# Vendor Tree (mt6895-common)
git clone https://github.com/XagaForge/android_vendor_xiaomi_mt6895-common.git vendor/xiaomi/mt6895-common --depth=1

echo "✅ Vendor trees cloned!"

---
## 🔶 Step 7: Clone Kernel Sources

In [ ]:
%%bash
cd /content/pos

echo "🔶 Cloning Kernel Sources..."

# Kernel Sources
git clone https://github.com/XagaForge/android_kernel_xiaomi_mt6895.git kernel/xiaomi/mt6895 --depth=1

echo "✅ Kernel sources cloned!"

---
## 🛠️ Step 8: Clone Hardware & MediaTek Repos

In [ ]:
%%bash
cd /content/pos

echo "🛠️ Cloning Hardware & MediaTek repos..."

# MediaTek IMS
rm -rf vendor/mediatek/ims 2>/dev/null || true
git clone https://github.com/XagaForge/android_vendor_mediatek_ims.git vendor/mediatek/ims --depth=1

# MediaTek SEPolicy
rm -rf device/mediatek/sepolicy_vndr 2>/dev/null || true
git clone https://github.com/XagaForge/android_device_mediatek_sepolicy_vndr.git device/mediatek/sepolicy_vndr --depth=1

# MediaTek Hardware
rm -rf hardware/mediatek 2>/dev/null || true
git clone https://github.com/XagaForge/android_hardware_mediatek.git hardware/mediatek --depth=1

# Xiaomi Hardware
rm -rf hardware/xiaomi 2>/dev/null || true
git clone https://github.com/XagaForge/android_hardware_xiaomi.git hardware/xiaomi --depth=1

echo "✅ Hardware repos cloned!"

---
## 📸 Step 9: Clone Optional Repos (Firmware & MiuiCamera)

In [ ]:
%%bash
cd /content/pos

echo "📸 Cloning Optional Repos..."

# Firmware (only needed when NOT building fastboot image)
git clone https://github.com/XagaForge/android_vendor_firmware.git vendor/firmware --depth=1

# MiuiCamera
git clone https://gitlab.com/priiii1808/proprietary_vendor_xiaomi_miuicamera-xaga.git vendor/xiaomi/miuicamera-xaga --depth=1

echo "✅ Optional repos cloned!"

---
## 🔧 Step 10: Apply Required Patches (wpa_supplicant)

In [ ]:
%%bash
cd /content/pos

echo "🔧 Applying required patches..."

# Navigate to wpa_supplicant_8 directory
cd external/wpa_supplicant_8

# Apply MediaTek changes for wpa_supplicant_8
echo "Applying MediaTek changes patch..."
git fetch https://github.com/Nothing-2A/android_external_wpa_supplicant_8 39200b6c7b1f9ff1c1c6a6a5e4cd08c6f526d048 --depth=1
git cherry-pick 39200b6c7b1f9ff1c1c6a6a5e4cd08c6f526d048 --no-commit || true

# Apply WAPI enablement patch
echo "Applying WAPI enablement patch..."
git fetch https://github.com/Nothing-2A/android_external_wpa_supplicant_8 37a6e255d9d68fb483d12db550028749b280509b --depth=1
git cherry-pick 37a6e255d9d68fb483d12db550028749b280509b --no-commit || true

cd /content/pos

echo "✅ Patches applied!"

---
## 🏗️ Step 11: Setup Build Environment & Start Build

In [ ]:
%%bash
cd /content/pos

# Setup ccache for faster builds
export USE_CCACHE=1
export CCACHE_EXEC=$(which ccache)
export CCACHE_DIR=/content/drive/MyDrive/ccache
mkdir -p $CCACHE_DIR
ccache -M 50G

echo "✅ ccache configured!"

# Source build environment
source build/envsetup.sh

# Lunch for xaga device (PixelOS uses aosp_xaga-userdebug format)
lunch aosp_xaga-userdebug

echo "✅ Build environment ready!"
echo "🏗️ Starting build... This will take several hours!"

In [ ]:
%%bash
cd /content/pos

# Source again (required for each new cell)
source build/envsetup.sh
lunch aosp_xaga-userdebug

# Start the build
# Use mka for parallel builds or m for single-threaded
mka bacon -j$(nproc --all)

---
## 💾 Step 12: Copy Output to Google Drive

In [ ]:
%%bash
# Create output directory in Google Drive
mkdir -p /content/drive/MyDrive/PixelOS_Output

# Copy the built ROM to Google Drive
cp /content/pos/out/target/product/xaga/PixelOS*.zip /content/drive/MyDrive/PixelOS_Output/
cp /content/pos/out/target/product/xaga/recovery.img /content/drive/MyDrive/PixelOS_Output/ 2>/dev/null || true
cp /content/pos/out/target/product/xaga/boot.img /content/drive/MyDrive/PixelOS_Output/ 2>/dev/null || true

echo "✅ Build output copied to Google Drive!"
echo "📁 Location: /content/drive/MyDrive/PixelOS_Output/"
ls -la /content/drive/MyDrive/PixelOS_Output/

---
## 📝 Notes & Troubleshooting

### Common Issues:
1. **Runtime disconnects**: Colab may disconnect during long builds. Consider using a keep-alive script.
2. **Out of RAM**: Use Colab Pro for more RAM, or reduce parallel jobs (-j4 instead of -j$(nproc)).
3. **Out of disk space**: The full source is ~200GB+. Make sure to use deep clones (--depth=1).
4. **Patch conflicts**: If patches fail to apply, you may need to manually resolve conflicts.

### Device Info:
- **Codename**: xaga
- **Devices**: POCO X4 GT / Redmi K50i / Redmi Note 11T Pro(+)
- **SoC**: MediaTek MT6895 (Dimensity 8100)

### Source Repos:
- [PixelOS Manifest](https://github.com/PixelOS-AOSP/android_manifest)
- [XagaForge Trees](https://github.com/XagaForge)

---
## 🔁 Optional: Keep Colab Alive Script
Run this in a separate browser tab's console to prevent disconnection:

In [ ]:
# Run this cell to display the keep-alive JavaScript code
# Copy and paste this into your browser's developer console (F12 -> Console)

keep_alive_script = '''
// Paste this in browser console to prevent Colab disconnection
function KeepColabAlive() {
    console.log("Keeping Colab alive...");
    document.querySelector("colab-connect-button").click();
}
setInterval(KeepColabAlive, 60000); // Click every 60 seconds
'''

print(keep_alive_script)